In [ ]:
import utility_module_2_v_3 as uu
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import numpy.ma as ma
from numpy import meshgrid
from mpl_toolkits.basemap import Basemap
from tqdm import tqdm

import geopandas as gpd
from shapely.geometry  import Point
from shapely.geometry import shape
import seaborn as sns


In [ ]:

output_figname = ''

path_out_P = '/home/vikram/BCSD_data/' + 'US_P_BCSD'+output_figname+'/'
path_out_T = '/home/vikram/BCSD_data/' + 'US_T_BCSD'+output_figname+'/'


# init

In [ ]:
gcm_names=['MIROC_ESM','CanESM2','HadGEM_AO','GFDL_CM3']
gcm_name  =  gcm_names[1]

# Import data

In [ ]:


metadata=np.load(path_out_P+ '/pre_metadata'+gcm_name+'.npy', allow_pickle=True)
st=metadata.item()['st']
en=metadata.item()['en']
mask_fine_obs=metadata.item()['mask']

lat_obs=metadata.item()['lat_obs']
lon_obs=metadata.item()['lon_obs']
lat_gcm_0=metadata.item()['lat_gcm']
lon_gcm_0=metadata.item()['lon_gcm']

tas_obs=np.load(path_out_T+ '/tas_obs.npy')

tas_data_all=np.empty([4,tas_obs.shape[0],tas_obs.shape[1],tas_obs.shape[2]])

tas_data_all[0]=tas_obs


tas_gcm=np.load(path_out_T+ '/tas_gcm_future_'+gcm_name+'.npy')

metadata=np.load(path_out_P+ '/pre_metadata'+gcm_name+'.npy', allow_pickle=True)
lat_gcm=metadata.item()['lat_gcm']
lon_gcm=metadata.item()['lon_gcm']

tas_gcm_fine=np.zeros((tas_gcm.shape[0],tas_obs.shape[1],tas_obs.shape[2]))
for i in range(tas_gcm.shape[0]):
    tas_gcm_fine[i,:,:]=uu.regrid(tas_gcm[i,:,:],lat_gcm,lon_gcm,lat_obs,lon_obs)

tas_data_all[1]        = np.concatenate((tas_gcm_fine  , np.nan * np.zeros((120 ,tas_obs.shape[1],tas_obs.shape[2]))),axis = 0)
tas_data_all[2] = np.concatenate((np.load(path_out_T+ '/future_BCSD_outputs_tas_'+gcm_name+'_cru.npy')  ,
                                               np.nan * np.zeros((120 ,tas_obs.shape[1],tas_obs.shape[2]))),axis = 0)


        
        

pre_obs=np.load(path_out_P+ '/pre_obs.npy')
pre_data_all=np.empty([4,pre_obs.shape[0],pre_obs.shape[1],pre_obs.shape[2]])

pre_data_all[0]=pre_obs


pre_gcm=np.load(path_out_P+ '/pre_gcm_future_'+gcm_name+'.npy')

metadata=np.load(path_out_P+ '/pre_metadata'+gcm_name+'.npy', allow_pickle=True)

lat_gcm=metadata.item()['lat_gcm']
lon_gcm=metadata.item()['lon_gcm']

pre_gcm_fine=np.zeros((pre_gcm.shape[0],pre_obs.shape[1],pre_obs.shape[2]))

for i in range(pre_gcm.shape[0]):
    pre_gcm_fine[i,:,:]=uu.regrid(pre_gcm[i,:,:],lat_gcm,lon_gcm,lat_obs,lon_obs)

pre_data_all[1]= np.concatenate((pre_gcm_fine  , np.nan * np.zeros((120 ,pre_obs.shape[1],pre_obs.shape[2]))),axis = 0)
pre_data_all[2]= np.concatenate((np.load(path_out_P+ '/future_BCSD_outputs_pre_'+gcm_name+'_cru.npy')  ,
                                                np.nan * np.zeros((120 ,pre_obs.shape[1],pre_obs.shape[2]))),axis = 0)


        



In [ ]:
obs_data = np.reshape(tas_data_all[0],(12,-1,52,112), order = 'F')
esm_data = np.reshape(tas_data_all[1],(12,-1,52,112), order = 'F')
var_obs =np.std(obs_data,axis=1)[:,np.newaxis,:,:]
mean_obs = np.mean(obs_data,axis=1)[:,np.newaxis,:,:]
var_esm =np.std(esm_data,axis=1)[:,np.newaxis,:,:]
mean_esm =  np.mean(esm_data,axis=1)[:,np.newaxis,:,:]
tas_data_all[3]   =  ((esm_data  -mean_esm)/var_esm  * var_obs  +  mean_obs).reshape((-1,52,112), order = 'F')


obs_data = np.reshape(pre_data_all[0],(12,-1,52,112), order = 'F')
esm_data = np.reshape(pre_data_all[1],(12,-1,52,112), order = 'F')

var_obs =np.std(obs_data,axis=1)[:,np.newaxis,:,:]
mean_obs = np.mean(obs_data,axis=1)[:,np.newaxis,:,:]
var_esm =np.std(esm_data,axis=1)[:,np.newaxis,:,:]
mean_esm =  np.mean(esm_data,axis=1)[:,np.newaxis,:,:]

pre_data_all[3]   =  ((esm_data  -mean_esm)/var_esm  * var_obs  +  mean_obs).reshape((-1,52,112), order = 'F')





In [ ]:
pre_6_monthly=np.empty([pre_data_all.shape[0],pre_data_all.shape[1]-5,pre_data_all.shape[2],pre_data_all.shape[3]])

for i in range(pre_data_all.shape[0]):
  pre_6_monthly[i,:,:,:]=(pre_data_all[i,0:-5,:,:]+
                          pre_data_all[i,1:-4,:,:]+
                          pre_data_all[i,2:-3,:,:]+
                          pre_data_all[i,3:-2,:,:]+
                          pre_data_all[i,4:-1,:,:]+
                          pre_data_all[i,5:,:,:])/6


# SPI

In [ ]:
import scipy.stats

def calc_spi_nonparam(data):
  N=data.shape[0]
  index_ts=np.arange(N)  

  data_non_zeros=(data[data>0])
  index_non_zeros=index_ts[data>0]
  N_non_zeros=len(data_non_zeros)
  index_sorted=  np.argsort(data_non_zeros)
  ranks = np.empty_like(index_sorted)
  ranks[index_sorted] = np.arange(N_non_zeros)

  p_gringorten=(ranks-0.44)/(N_non_zeros+0.12)
  p_g_all=np.zeros_like(data)
  p_g_all[data>0]=p_gringorten


  count_zeros=(data[data==0].shape[0])
  count_total=(data.shape[0])
  p_zeros=count_zeros/count_total

  cdf2 = p_zeros + (1 - p_zeros) * p_g_all
  SPI= scipy.stats.norm.ppf(cdf2)
  return SPI



In [ ]:
N1=pre_obs.shape[1]
N2=pre_obs.shape[2]

SPI_monthly_np=np.empty(pre_6_monthly.shape)

for i in tqdm(range(4)):
  for n1 in range(N1):
    for n2 in range(N2):
      SPI_monthly_np[i,:,n1,n2]=calc_spi_nonparam(pre_6_monthly[i,:,n1,n2].flatten())


# Check imported data sn SPI calclulation

In [ ]:
print(SPI_monthly_np.shape,
     tas_data_all.shape)

In [ ]:
plt.pcolor(pre_data_all[2,0,:,:])
plt.colorbar()

# Calculate delta 

In [ ]:

period_0=[1902,1951] # ind1=(period_0[0]-st)*12 ++ ind2=(period_0[1]-st+1)*12
period_1=[1956,2005] # ind1=(period_1[0]-st)*12 ++ ind2=(period_1[1]-st+1)*12
N1=tas_data_all.shape[2]
N2=tas_data_all.shape[3]
## OBS  ##
tas_period_0=np.zeros([tas_data_all.shape[0],50*12,N1,N2])
tas_period_1=np.zeros([tas_data_all.shape[0],50*12,N1,N2])
delta_tas=np.zeros([tas_data_all.shape[0],N1,N2])
spi_period_0=np.zeros([SPI_monthly_np.shape[0],50*12,N1,N2])
spi_period_1=np.zeros([SPI_monthly_np.shape[0],50*12,N1,N2])

for i in range(4):
  tas_period_0[i,:,:,:]=tas_data_all[i,(period_0[0]-st)*12:(period_0[1]-st+1)*12,:,:]
  tas_period_1[i,:,:,:]=tas_data_all[i,(period_1[0]-st)*12:(period_1[1]-st+1)*12,:,:]
  t0=tas_period_0[i,:,:,:]
  t1=tas_period_1[i,:,:,:]
  
  #print(t0.shape,t1.shape)

  mean_tas_period_0=np.mean(t0,axis=0)
  mean_tas_period_1=np.mean(t1,axis=0)
  #print(mean_tas_period_0.shape,mean_tas_period_1.shape)

  delta_tas[i,:,:]=mean_tas_period_1-mean_tas_period_0
  #print(delta_tas.shape)

  spi_period_0[i,:,:,:]=SPI_monthly_np[i,(period_0[0]-st)*12-5:(period_0[1]-st+1)*12-5,:,:]
  spi_period_1[i,:,:,:]=SPI_monthly_np[i,(period_1[0]-st)*12-5:(period_1[1]-st+1)*12-5,:,:]

  #print(spi_period_0.shape)
  #print(spi_period_1.shape)



In [ ]:
delta_tas_d0=np.zeros((13,N1,N2))
delta_tas_d1=np.zeros((13,N1,N2))

delta_spi_freq_sp1=np.zeros((13,N1,N2))
delta_spi_freq_sp2=np.zeros((13,N1,N2))
delta_spi_freq_sp3=np.zeros((13,N1,N2))

for i in tqdm(range(4)):
  for n1 in range(N1):
    for n2 in range(N2):

      spi0=spi_period_0[i,:,n1,n2]
      spi1=spi_period_1[i,:,n1,n2]

      tp0=tas_period_0[i,:,n1,n2]
      tp1=tas_period_1[i,:,n1,n2]

      tas_d0_p0=np.nanmean(tp0[np.all(np.array([spi0<=-0.5]),axis=0)])
      tas_d1_p0=np.nanmean(tp0[np.all(np.array([spi0<=-0.8]),axis=0)])

      tas_d0_p1=np.nanmean(tp1[np.all(np.array([spi1<=-0.5]),axis=0)])
      tas_d1_p1=np.nanmean(tp1[np.all(np.array([spi1<=-0.8]),axis=0)])
      
      d__01=np.all(np.array([spi0<=-0.5]),axis=0).sum()
      d__02=np.all(np.array([spi0<=-0.8]),axis=0).sum()
      d__03=np.all(np.array([spi0<=-1.3]),axis=0).sum()
      
      d__11=np.all(np.array([spi1<=-0.5]),axis=0).sum()
      d__12=np.all(np.array([spi1<=-0.8]),axis=0).sum()
      d__13=np.all(np.array([spi1<=-1.3]),axis=0).sum()

      del_spi_1=(d__11-d__01)/50
      del_spi_2=(d__12-d__02)/50
      del_spi_3=(d__13-d__03)/50
      
      delta_spi_freq_sp1[i,n1,n2]=del_spi_1
      delta_spi_freq_sp2[i,n1,n2]=del_spi_2
      delta_spi_freq_sp3[i,n1,n2]=del_spi_3

      delta_tas_d0[i,n1,n2]=tas_d0_p1-tas_d0_p0
      delta_tas_d1[i,n1,n2]=tas_d1_p1-tas_d1_p0


# Plotting change

## Creating mask

In [ ]:

fo=gpd.read_file('/home/vikram/BCSD_Shapefiles/Us_boundary.shp')
fo.plot()
shp_geom = shape(fo.geometry[0])

xxc, yyc = np.meshgrid(lon_obs.data-360,lat_obs.data)
mask_shape=np.empty_like(xxc)

N1=xxc.shape[0]
N2=xxc.shape[1]
for lat_i in range(N1):
  for lon_i in range(N2):
    xi=xxc[lat_i,lon_i]
    yi=yyc[lat_i,lon_i]
    m1= (shp_geom.contains(Point(xi, yi)) or shp_geom.touches(Point(xi, yi)))
    mask_shape[lat_i,lon_i]=not m1


## Figure 5

In [ ]:
def map_plot_cyl_final(data_lat_x_lon,lat_cen,lon_cen,map_bounds,clab,y_limit,ax,mp_spacing=10,subplot_iter=1,loff=1,coff=1,ex='both'):

  lat_box=uu.cen_to_box(lat_cen)
  lon_box=uu.cen_to_box(lon_cen)

  map = Basemap(projection='cyl',resolution='l',
                llcrnrlon=map_bounds[0], 
                urcrnrlat=map_bounds[1],
                urcrnrlon=map_bounds[2], 
                llcrnrlat=map_bounds[3])


  xx, yy = meshgrid(lon_box,lat_box )
  #return  dimenion of lat * lon
  map.pcolor(xx, yy, data_lat_x_lon,cmap='RdBu_r')
  map.drawmeridians(np.arange(-180,180,mp_spacing),labels=[0,0,0,1], linewidth=1.0) #longitudes
  if loff==0:
    map.drawparallels(np.arange(-90,90,mp_spacing),labels=[1,0,0,0], linewidth=1.0) #latitudes
  else:
    map.drawparallels(np.arange(-90,90,mp_spacing),labels=[0,0,0,0], linewidth=1.0) #latitudes
  cbar = map.colorbar(extend=ex)
  #cbar.ax.set_yticklabels(['0','1','2','>3'])
  cbar.set_label(clab, rotation=90)
  plt.clim(y_limit)

  if coff==1:
    cbar.remove()

  #map.drawcountries(linewidth=1)
  map.readshapefile('/home/vikram/BCSD_Shapefiles/Us_boundary','UU',linewidth=2.0)

  map.drawcoastlines(linewidth=1)
  lxx=0.02
  lyy=1.02
  lcol='k'
  lsize=24
  lfamily='Arial'
  lweight=800
 # y_limit=[0,30]
  #clab='days'
  
  plt.text(lxx, lyy, chr(subplot_iter+96), horizontalalignment='left',verticalalignment='bottom', transform=ax.transAxes,fontweight=lweight,
           fontfamily=lfamily,fontsize=lsize,color=lcol)

In [ ]:
data_bounds=[ -125, 50, -70, 25]
gcm_iter=1
Dl=[0,1,2,3]
gcm_name=gcm_names[gcm_iter]
dapa=[delta_tas[0,:,:],
  delta_tas[1,:,:],
  delta_tas[3,:,:],
  delta_tas[2,:,:],
  delta_spi_freq_sp2[0,:,:],
  delta_spi_freq_sp2[1,:,:],
  delta_spi_freq_sp2[3,:,:],
  delta_spi_freq_sp2[2,:,:],
  delta_tas_d1[0,:,:],
  delta_tas_d1[1,:,:],
  delta_tas_d1[3,:,:],
  delta_tas_d1[2,:,:]]
y_limits=[[-1.5,1.5],
         [-1.5,1.5],
         [-1.5,1.5]]

y_limits=[[-3,3],
         [-3,3],
         [-3,3]]


lofff=[0,1,1,1,0,1,1,1,0,1,1,1]
cofff=[1,1,1,0,1,1,1,0,1,1,1,0]
titlesx=['Observation','Raw '+gcm_name,'Standardisation '+gcm_name,'BCSD '+gcm_name]
titlesy=['$\Delta$T','$\Delta_{freq}$D1','$\Delta$T|D1']
labels=['$\degree$C','drought months/year','$\degree$C']

ex='both'
fig=plt.figure(figsize=(20,8))

for rr in range(3):
  for cc in range(4):
    iter1=rr*4+cc
    iter2=rr*3+cc
    ax=plt.subplot(3,4,iter1+1)
    #plt.pcolor(ma.masked_array(dapa[rr*4+cc], mask=mask))
    map_plot_cyl_final(ma.masked_array(dapa[iter1], mask=mask_shape),
                       lat_obs,lon_obs-360,data_bounds,labels[rr],y_limits[rr],
                       ax,mp_spacing=10,subplot_iter=iter1+1,loff=lofff[iter1],
                       coff=cofff[iter1],ex=ex)
    plt.title(titlesx[cc])


fig.text(0.08, 0.78, titlesy[0], va='center', ha='center', rotation='vertical', fontsize=16)
fig.text(0.08, 0.5, titlesy[1], va='center', ha='center', rotation='vertical', fontsize=16)
fig.text(0.08, 0.25, titlesy[2], va='center', ha='center', rotation='vertical', fontsize=16)
# plt.savefig('/home/vikram/Dropbox/BCSD Work/03_Figures/Figures/Figure5/with_std_extended_'+gcm_name+output_figname+'.svg',
#            facecolor = 'w')

plt.savefig('/home/vikram/Dropbox/BCSD Work/03_Figures/Figure_new_auroop/Figure3/future_part_1_with_std_extended_'+gcm_name+output_figname+'.svg',
           facecolor = 'w')



# S score

In [ ]:
def S_score_2D(data_ref, data_model,mask_inf):
    # mask is true for exterior values
    data_1 = data_ref
    data_2 = data_model 
    ind_1 = np.isnan(data_1)
    ind_2 = np.isnan(data_2)
    data_1[ind_2] = np.nan
    data_2[ind_1] = np.nan
    data_1[mask_inf == True] = np.nan
    data_2[mask_inf == True] = np.nan

    data_1 = data_1.flatten()
    data_2 = data_2.flatten()
    
    data_1 =  np.delete(data_1,np.isnan(data_1))
    data_2 =  np.delete(data_2,np.isnan(data_2))
    R = np.corrcoef(data_1,data_2)[0,1]

    SDR = np.std(data_2)/np.std(data_1)

    S_score = (1+R)**4  / ( 4 * (SDR + 1 / SDR)**2 )

    
    return S_score

In [ ]:
plt.pcolor(mask_shape)
plt.colorbar()

In [ ]:
plt.pcolor(delta_tas_d1[1,:,:])

In [ ]:
D=4
N_index= 3
S_score_matrix = np.zeros((N_index,D))
for itr,index in enumerate([delta_tas,delta_spi_freq_sp2,delta_tas_d1]):
    for di,d in enumerate([0,1,3,2]):
        S_score_matrix[itr,di] = (S_score_2D(index[1,:,:],index[d,:,:],mask_shape))
        
titlesy=['$\Delta$T','$\Delta_{freq}$D1','$\Delta$T|D1']
        
h=sns.heatmap(S_score_matrix , linewidth = 2 , cmap = 'Reds', 
          annot=True,
          fmt=".2f", square=True,
          xticklabels=['O','G','S','B'],
          yticklabels=titlesy,vmin=0,vmax=1,
             
          cbar_kws={'label': 'S score','extend':None},cbar = True)


plt.savefig('/home/vikram/Dropbox/BCSD Work/03_Figures/Figures/Figure5/with_std_Score_extended_wrt_gcm_'+gcm_name+output_figname+'.svg')

plt.savefig('/home/vikram/Dropbox/BCSD Work/03_Figures/Figure_new_auroop/Figure3/with_std_Score_extended_wrt_gcm_'+gcm_name+output_figname+'.svg',
           facecolor = 'w')


plt.figure()
D=4
N_index= 3
S_score_matrix = np.zeros((N_index,D))
for itr,index in enumerate([delta_tas,delta_spi_freq_sp2,delta_tas_d1]):
    for d in [0,1,3,2]:
        S_score_matrix[itr,d] = (S_score_2D(index[0,:,:],index[d,:,:],mask_shape))
        
titlesy=['$\Delta$T','$\Delta_{freq}$D1','$\Delta$T|D1']
        
h=sns.heatmap(S_score_matrix , linewidth = 2 , cmap = 'Reds', 
          annot=True,
          fmt=".2f", square=True,
          xticklabels=['O','G','S','B'],
          yticklabels=titlesy,vmin=0,vmax=1,
             
          cbar_kws={'label': 'S score','extend':None},cbar = True)

plt.savefig('/home/vikram/Dropbox/BCSD Work/03_Figures/Figure_new_auroop/Figure3/Score_extended_wrt_obs_'+gcm_name+output_figname+'.svg',
           facecolor = 'w')

# plt.savefig('/home/vikram/Dropbox/BCSD Work/03_Figures/Figures/Figure5/Score_extended_wrt_obs_'+gcm_name+output_figname+'.svg')

# cbar_kws={'label': '°C','extend':'max'},cbar

In [ ]:
path_out_P

# Other

## Other


In [ ]:


def plot_hmap(data_corr,ylim,titles,fmt,cmap):
  fig=plt.figure(figsize=(14,8))
  S=data_corr[0].shape[0]
  for iii in range(3):
    for subs in range(S):
      ax=plt.subplot(3,4,iii*4+subs+1)
      
      data_set = data_corr[iii][subs,:,:]
      
      if (subs<S-1):
        cbarI=False
      else:
        cbarI=True

      h=sns.heatmap(data_set , linewidth = 2 , cmap = cmap, 
                  annot=True,
                  fmt=fmt ,square=True,
                  xticklabels=titles,
                  yticklabels=titles,vmin=ylim[iii][0],vmax=ylim[iii][1],
                    cbar_kws={'label': '°C','extend':'max'},cbar=cbarI)
      if iii==0:
        plt.title(region_shapes['NAME'][subs])
        
      lxx=-0.20
      lyy=0.8
      lcol='k'
      lsize=24
      lfamily='Tahoma'
      lweight=800
      
      plt.text(lxx, lyy,  chr(iii*4+subs+1+96), horizontalalignment='right',verticalalignment='bottom', transform=ax.transAxes,fontweight=lweight,
              fontsize=lsize,color=lcol)    

  fig.text(0.08, 0.78, titlesy[0], va='center', ha='center', rotation='vertical', fontsize=16)
  fig.text(0.08, 0.5, titlesy[1], va='center', ha='center', rotation='vertical', fontsize=16)
  fig.text(0.08, 0.25, titlesy[2], va='center', ha='center', rotation='vertical', fontsize=16)
  fig.suptitle(gcm_name)
  plt.savefig('/home/vikram/Dropbox/BCSD Work/03_Figures/Figures/Figure5/extended_rmse_'+gcm_name+'.svg')
  plt.show()

In [ ]:
titles=['Obs','GCM','BC-G','BC-R']


In [ ]:
plot_hmap([ind1h,ind2h,ind3h],[[0,1.5],[0,1.5],[0,1.5]],titles,".1f",'Reds')

### RMSE SHort

In [ ]:


def plot_hmap_short(data_corr,ylim,titles,fmt,cmap):
  titlesy=['$\Delta$T','$\Delta_{freq}$D1','$\Delta$T|D1']
  fig=plt.figure(figsize=(12,2))
  S=data_corr[0].shape[0]
  for iii in range(3):
      ax=plt.subplot(1,3,iii+1)
      
      data_set=np.zeros((4,3))
      for subs in range(S):
        data_set[subs,:]= data_corr[iii][subs,1:,0]

      h=sns.heatmap(data_set.T , linewidth = 2 , cmap = cmap, 
                  annot=True,
                  fmt=fmt ,square=True,
                  xticklabels=titles[0],
                  yticklabels=titles[1],vmin=ylim[iii][0],vmax=ylim[iii][1],
                  cbar_kws={'label': labels[iii],'extend':'max'})
        
      lxx=0
      lyy=1
      lcol='k'
      lsize=24
      lfamily='Tahoma'
      lweight=800
      
      plt.text(lxx, lyy,  chr(iii+1+96), horizontalalignment='right',verticalalignment='bottom', transform=ax.transAxes,fontweight=lweight,
              fontsize=lsize,color=lcol)    
      plt.title(titlesy[iii])

  fig.text(0.5, -0.1, gcm_name, va='center', ha='center', fontsize=16)
  plt.savefig('/home/vikram/Dropbox/BCSD Work/03_Figures/Figures/Figure5/extended_rmse_ss'+gcm_name+'.svg')
  plt.show()

In [ ]:
titlesy=['GCM','BC-G','BC-R']
titlesx=['N','S','W','MW']
labels=['$\degree$C','drought months/year','$\degree$C']
titles=[titlesx,titlesy]

In [ ]:
plot_hmap_short([ind1h,ind2h,ind3h],[[0,1.5],[0,1.5],[0,1.5]],titles,".1f",'Reds')

## Final short

In [ ]:
gcm_iter=3
Dl=[0,1+gcm_iter,5+gcm_iter,9+gcm_iter]
gcm_name=gcm_names[gcm_iter]
dapa=[delta_tas_d1[Dl[0],:,:],
  delta_tas_d1[Dl[1],:,:],
  delta_tas_d1[Dl[2],:,:],
  delta_tas_d1[Dl[3],:,:]]

y_limit=[-1.5,1.5]
labels=['$\degree$C']
lofff=[0,1,0,1]
cofff=[0,0,0,0]
titlesx=['Observation','Raw '+gcm_name,'BCSD '+gcm_name,'BCSD Random']
titlesy=['$\Delta$T|D1']
ex='both'
fig=plt.figure(figsize=(16,8))

for rr in range(2):
  for cc in range(2):
    iter=rr*2+cc
    ax=plt.subplot(2,2,iter+1)
    #plt.pcolor(ma.masked_array(dapa[rr*4+cc], mask=mask))
    map_plot_cyl_final(ma.masked_array(dapa[iter], mask=mask_shape),lat_obs,lon_obs-360,data_bounds,labels[0],y_limits[0],ax,mp_spacing=10,subplot_iter=iter+1,loff=lofff[iter],coff=cofff[iter],ex=ex)
    plt.title(titlesx[iter])


fig.text(0.08, 0.5, titlesy[0], va='center', ha='center', rotation='vertical', fontsize=16)
fig.suptitle(gcm_name)
plt.savefig('/content/drive/MyDrive/Figures/Figure5/brief_'+gcm_name+'.svg')

In [ ]:
data_bounds=[ 360-125, 50, 360-70, 25]

lab_list=['Obs','GCM0*','GCM1*','GCM2*','GCM3*','BCSD-GCM0','BCSD-GCM1','BCSD-GCM2','BCSD-GCM3','BCSD-RND0','BCSD-RND1','BCSD-RND2','BCSD-RND3']
lweight=500
lsize=20
lcol='k'
sx=1
sy=3
fig=plt.figure(figsize=(25,60))
ylim=[250,330]
for ii in range(sx):
  ind=ii*3
  ax=plt.subplot(sx,sy,ii*3+1)
  map_plot_cyl_change(delta_tas[ii],lat_obs,lon_obs,data_bounds,'',ylim,ax,mp_spacing=10,subplot_iter=ii+1,loff=(ind+1)%3,coff=(ind+1)%3)
  plt.title(lab_list[ii]+': $\Delta$T',fontweight=lweight,fontsize=lsize,color=lcol)

  ind=ii*3+1
  ax=plt.subplot(sx,sy,ii*3+1+1) 
  tt0=tas_period_0[ii,:,:,:]

  map_plot_cyl_change(tt0[0,:,:],lat_obs,lon_obs,data_bounds,'',ylim,ax,mp_spacing=10,subplot_iter=ii+1,loff=(ind+1)%3,coff=(ind+1)%3)
  plt.title(lab_list[ii]+': $\Delta$T|D0',fontweight=lweight,fontsize=lsize,color=lcol)
  ind=ii*3+1
  tt1=tas_period_1[ii,:,:,:]
  ax=plt.subplot(sx,sy,ii*3+2+1)
  map_plot_cyl_change(np.nanmean(tt1,axis=0),lat_obs,lon_obs,data_bounds,'',ylim,ax,mp_spacing=10,subplot_iter=ii+1,loff=(ind+1)%3,coff=(ind+1)%3)
  plt.title(lab_list[ii]+': $\Delta$T|D1',fontweight=lweight,fontsize=lsize,color=lcol)

plt.suptitle("Change in Annual Mean: (1902-1951) to (1955-2005)",fontsize=24,color='k')



In [ ]:
data_bounds=[ 360-125, 50, 360-70, 25]

lab_list=['Obs','GCM0*','GCM1*','GCM2*','GCM3*','BCSD-GCM0','BCSD-GCM1','BCSD-GCM2','BCSD-GCM3','BCSD-RND0','BCSD-RND1','BCSD-RND2','BCSD-RND3']
lweight=500
lsize=20
lcol='k'
sx=13
sy=3
fig=plt.figure(figsize=(25,60))
ylim=[250,330]
for ii in range(sx):
  ind=ii*3
  ax=plt.subplot(sx,sy,ii*3+1)
  map_plot_cyl_change(delta_tas[ii],lat_obs,lon_obs,data_bounds,'',ylim,ax,mp_spacing=10,subplot_iter=ind+1,loff=(ind+1)%3,coff=(ind+1)%3)
  plt.title(lab_list[ii]+': $\Delta$T',fontweight=lweight,fontsize=lsize,color=lcol)

  ind=ii*3+1
  ax=plt.subplot(sx,sy,ii*3+1+1) 
  tt0=spi_period_0[ii,:,:,:]

  map_plot_cyl_change(tt0[0,:,:],lat_obs,lon_obs,data_bounds,'',ylim,ax,mp_spacing=10,subplot_iter=ind+1,loff=(ind+1)%3,coff=(ind+1)%3)
  plt.title(lab_list[ii]+': $\Delta$T|D0',fontweight=lweight,fontsize=lsize,color=lcol)
  ind=ii*3+2
  tt1=spi_period_1[ii,:,:,:]
  ax=plt.subplot(sx,sy,ii*3+2+1)
  map_plot_cyl_change(np.nanmean(tt1,axis=0),lat_obs,lon_obs,data_bounds,'',ylim,ax,mp_spacing=10,subplot_iter=ind+1,loff=(ind+1)%3,coff=(ind+1)%3)
  plt.title(lab_list[ii]+': $\Delta$T|D1',fontweight=lweight,fontsize=lsize,color=lcol)

plt.suptitle("Change in Annual Mean: (1902-1951) to (1955-2005)",fontsize=24,color='k')

